In [1]:
#%%
import os
import re
import sys
import time
import json

import numpy as np
import pandas as pd
from pandas.core.indexes.base import Index

from tqdm import tqdm


#%%
# !pip install chromedriver_autoinstaller
from bs4 import BeautifulSoup
import chromedriver_autoinstaller

import selenium
from selenium import webdriver
from selenium.webdriver import ActionChains

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select, WebDriverWait

from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, ElementNotInteractableException

# from html_table_parser import parser_functions as parser

pd.set_option('mode.chained_assignment', None)

In [2]:
from pyvirtualdisplay import Display
display = Display(visible=0, size=(1920,1080))
display.start()

In [3]:
#%%
url = 'https://www.echemportal.org/echemportal/property-search'

option = webdriver.ChromeOptions()
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
option.add_argument('window-size=1920,1080')

driver_path = '/home/rt/chromedriver'
driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)
driver.implicitly_wait(3)
driver.get(url)


#@id는 F12에서 id라고 뜨는 위치다. 그리고 /div는 그 뒤에 몇개의 div가 있냐이고, a[2]는 a _ngcontent-c13이 2개인데, 그중에 2번째꺼를 의미
deselect_path = '//*[@id="datasources-panel-1"]/div/div/div/a[2]'
driver.find_element_by_xpath(deselect_path).click()
time.sleep(0.5)

#위에꺼의 2번째에 보면 div가 아니라 echem-search-sources란 창이 하나 더 있다. 찾기 어렵네 ㄷㄷ 그래서 쭉 찾아보면 div들이 있는 것 중 2번째를 선택하고, 그 안에 체크박스가 있고, div
echa_path = '//*[@id="datasources-panel-1"]/div/echem-search-sources/div/div/div/div[2]/echem-checkbox/div'
driver.find_element_by_xpath(echa_path).click()
time.sleep(0.5)

# add Query Block 버튼
query_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[1]/echem-query-builder/div[2]/div/button'
driver.find_element_by_xpath(query_path).click()
time.sleep(0.5)

# Toxicological information 버튼
tox_button_path = '//*[@id="QU.SE.7-toxicological-information-header"]/div/div[1]/button'
driver.find_element_by_xpath(tox_button_path).click()
time.sleep(0.5)

# repeated-dose
genetic_path = '//*[@id="QU.SE.7.5-repeated-dose-toxicity-header"]/div/div[2]'
driver.find_element_by_xpath(genetic_path).click()
time.sleep(0.5)

# header 하위 3개의 창
#inhalation
inh_path = '//*[@id="QU.SE.7.5-repeated-dose-toxicity"]/div/div/div[2]/div[3]/button'
driver.find_element_by_xpath(inh_path).click()
time.sleep(0.5)

/tmp/ipykernel_4317/3005824369.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="/home/rt/chromedriver", chrome_options=chrome_options)


In [4]:
#옆에 새로운 Define이란 창이 열림
#맨 위의 Type of information
info_type_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[1]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(info_type_path).click()
time.sleep(0.5)

# Type 중에서 experimental study를 선택
experiment_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[3]'
driver.find_element_by_xpath(experiment_path).click()
time.sleep(0.5)

#Test guideline 선택
tg_path = '//*[@id="property_query-builder-panel-1"]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/echem-property-phrase-field[4]/div/div/div/ng-select/div/div'
driver.find_element_by_xpath(tg_path).click()
time.sleep(0.5)

#TG422경로
tg422_path = '/html/body/ng-dropdown-panel/div[2]/div[2]/div[16]'
driver.find_element_by_xpath(tg422_path).click()
time.sleep(0.5)

#이건 저장이니 안건드려도 됨
save_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/ngb-accordion[1]/div[2]/div[2]/div/echem-property-query-panel/div[2]/div[3]/echem-property-form/form/div/button[2]'
driver.find_element_by_xpath(save_path).click()
time.sleep(0.5)

#이것도 검색시작이므로 안건드려도 됨
search_path = '/html/body/echem-root/div/echem-substance-search-page/echem-substance-search-container/echem-substance-search/form/div/div[2]/div/button'
driver.find_element_by_xpath(search_path).click()

In [5]:
#%%
result_ = []

page_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]'
page_num = int(driver.find_element_by_xpath(page_num_path).text.split(' ')[-1])

In [6]:
start = time.time()
for p in range(1, page_num + 1):
#for p in range(1, 4):
    row_num_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr'
    row_num = len(driver.find_elements_by_xpath(row_num_path))
    
    row = tqdm(range(1, row_num + 1), file = sys.stdout)
    
    for i in row:
        src_dict = {}
        
        chem_path = '//*[@id="top"]/echem-substance-search-page/echem-property-search-results-container/echem-property-search-results/table/tbody/tr[%d]/td[3]/a'
        property_url = driver.find_element_by_xpath(chem_path % i).get_attribute('href')
        src_dict['link'] = property_url
        
        driver.execute_script('window.open('');')
        driver.switch_to.window(driver.window_handles[1])
        driver.get(property_url)
        time.sleep(1)
        
        try:
            accept_path = '/html/body/div[1]/div/div[2]/div[2]/button[1]'
            driver.find_element_by_xpath(accept_path).send_keys(Keys.ENTER)
            #driver.find_element_by_xpath(accept_path).click()
        except NoSuchElementException:
            pass
        
        try:
            src = driver.page_source
            # soup = BeautifulSoup(src, 'html.parser')
            src_dict['src'] = src
            
            result_.append(src_dict)
        
        except AttributeError:
            pass

        driver.close()
        driver.switch_to.window(driver.window_handles[0])
        
        row.set_postfix({'page': p})
    
    if p < page_num:
        next_page_path = '/html/body/echem-root/div/echem-substance-search-page/echem-property-search-results-container/echem-pagination/div/div[2]/a[3]'
        driver.find_element_by_xpath(next_page_path).click()
        time.sleep(1.5)
    
    p += 1


print(time.time() - start)


#%%
json.dump(result_, open('tg422_inhalation_source.json', 'w'))

100%|████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.55s/it, page=24]
368.90206003189087


## json전처리

In [7]:
import re
import json

import numpy as np
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup


#%%
with open('tg422_inhalation_source.json', 'r') as file:
    df = pd.DataFrame(json.load(file))

#%%
def remove_bracket(string):
    clean_string = re.sub('<.*?>', '', str(string))
    return clean_string


#%%
result_ = []

for i in tqdm(range(len(df))):
#for i in tqdm(range(100)):
    CE = np.nan
    EL = np.nan
    RH = np.nan
    CS = np.nan
    try: 
        soup = BeautifulSoup(df.src[i], 'html.parser')
        chem_dict = {}
        
        # chemical name
        chem_name = soup.find('div', attrs = {'id': 'SubstanceName'}).find_next('h1').text
        chem_dict['Chemical'] = chem_name


        # casrn
        casrn_tmp = soup.find('div', attrs = {'class': 'container'}).find_next('strong').text
        casrn = re.sub('\n|\t', '', casrn_tmp).split( )[-1]
        chem_dict['CasRN'] = casrn
        
        # Critical effects observed:
        admin_tmp = soup.find_all("dl", {"class" : "HorDL"})

        for j in range(len(admin_tmp)):
            if "<dt>Critical effects observed:" in str(admin_tmp[j]):
                CE = str(admin_tmp[j]).split('<dt>Critical effects observed:</dt><dd>')[1]
                CE = CE.split('</dd>')[0]
            chem_dict['Critical effects'] = CE
            
        # Basis for effect level:
        for j in range(len(admin_tmp)):
            if "<dt>Basis for effect level:" in str(admin_tmp[j]):
                EL = str(admin_tmp[j]).split('<dt>Basis for effect level:</dt><dd>')[1]
                EL = EL.split('</dd>')[0]
            chem_dict['effect level'] = EL
            
        # Relevant for humans:
        for l in range(len(admin_tmp)):
            if "<dt>Relevant for humans:" in str(admin_tmp[l]):
                RH = str(admin_tmp[l]).split('<dt>Relevant for humans:</dt><dd>')[1]
                RH = RH.split('</dd>')[0]
            chem_dict['Relevant_human'] = RH
            
        # Clinical signs:
        for j in range(len(admin_tmp)):
            if "<dt>Clinical signs:" in str(admin_tmp[j]):
                CS = str(admin_tmp[j]).split('<dt>Clinical signs:</dt><dd>')[1]
                CS = CS.split('</dd>')[0]
            chem_dict['Clinical_sign'] = CS
            
        # other results
        result_and_discussion = soup.find('h3', attrs={'id': 'sResultsAndDiscussion'})
        table_list = result_and_discussion.find_next_sibling('div').find_all('dl')
            
            
        for tab in table_list:
            chem_dict_ = chem_dict.copy()
            
            key = [re.sub(':', '', i.text).strip() for i in tab.find_all('dt')]
            value = [i.text.strip() for i in tab.find_all('dd')]
                                    
            if len(key) == len(value) and key[0] != '' and value[0] != 'Key result':
                result_dict = dict(zip(key, value))
                # result_dict = {key[i]: re.sub('<.*?>', '', cell.text).strip() for i, cell in enumerate(tab.find_all('dd'))}
            
            elif len(key) == len(value) and key[0] == '' and value[0] == 'Key result':
                result_dict = dict(zip(key[1:], value[1:]))
            
            elif len(key) != len(value) and key[0] == '' and value[0] == 'Key result':
                key = key[1:]
                value_ = value[1:len(key)] + ['. '.join(value[len(key):])]
                result_dict = dict(zip(key, value_))
            
            chem_dict_.update(result_dict)
            result_.append(chem_dict_)

    except AttributeError:
        #print(f'{i}th error')
        pass
    


# %%
result = pd.DataFrame(result_)

100%|█████████████████████████████████████████████████████████████████████████████████| 117/117 [00:04<00:00, 24.23it/s]


In [8]:
result

Chemical       CasRN  \
0                 1,1,1,3,5,5,5-heptamethyltrisiloxane   1873-88-7   
1    Gases (petroleum), light steam-cracked, butadi...  68955-28-2   
2    Distillates (petroleum), cracked stripped stea...  68477-39-4   
3    Distillates (petroleum), cracked stripped stea...  68477-39-4   
4                                       1-nitropropane    108-03-2   
..                                                 ...         ...   
160                                          But-1-ene    106-98-9   
161                                       Carbon black   1333-86-4   
162                                       Carbon black   1333-86-4   
163                                       Carbon black   1333-86-4   
164                                       Carbon black   1333-86-4   

    Critical effects                                       effect level  \
0      not specified  other: Based on accumulation of brown pigment ...   
1      not specified                                other: see 'Remark'   
2      not specified  other: equivalent to 7817 mg/m3,  no alteratio...   
3      not specified  other: equivalent to 7817 mg/m3,  no alteratio...   
4      not specified  other: 100 ppm was the highest concentration t...   
..               ...                                                ...   
160    not specified  other: there were no adverse effects at 8000 p...   
161              yes  other: histopathology; prolonged carbon black ...   
162              yes  other: histopathology; prolonged carbon black ...   
163              yes  other: histopathology; prolonged carbon black ...   
164              yes  other: histopathology; prolonged carbon black ...   

    Relevant_human                        Clinical_sign Dose descriptor  \
0              NaN                  no effects observed           NOAEC   
1              NaN                  no effects observed           NOAEC   
2              NaN                                  NaN           NOAEC   
3              NaN                                  NaN           NOAEC   
4              NaN                  no effects observed           NOAEL   
..             ...                                  ...             ...   
160            NaN                  no effects observed           NOAEC   
161  not specified  effects observed, treatment-related            NOEC   
162  not specified  effects observed, treatment-related            LOEC   
163  not specified  effects observed, treatment-related            NOEC   
164  not specified  effects observed, treatment-related           LOAEC   

                                         Effect level   Based on          Sex  \
0                                            400  ppm  test mat.  male/female   
1                  20  other: mg/L air (20,000 mg/m3)        NaN  male/female   
2                                          1 800  ppm        NaN         male   
3                                          1 800  ppm        NaN         male   
4                                48  ppm (analytical)        NaN          NaN   
..                                                ...        ...          ...   
160  8 000  other: ppm (18359 mg/m3, 18 mg/L) nominal        NaN  male/female   
161                                      1  mg/m³ air  test mat.       female   
162                                      7  mg/m³ air  test mat.       female   
163                                      7  mg/m³ air  test mat.       female   
164                                     50  mg/m³ air  test mat.       female   

                                Basis for effect level       Remarks  \
0    other: Based on accumulation of brown pigment ...           NaN   
1                                  other: see 'Remark'           NaN   
2    other: equivalent to 7817 mg/m3,  minor (-6%) ...           NaN   
3    other: equivalent to 7817 mg/m3,  no alteratio...   ototoxicity   
4                                                  NaN           NaN  

In [9]:
result['Critical effects'].value_counts()

not specified    137
no                17
yes               11
Name: Critical effects, dtype: int64

In [10]:
result['effect level'].value_counts()

other: decreased white blood cell count, percentage of lymphocytes and femoral marrow cellularity at 960 mg/m3                                                                                                                             26
other: highest concentration tested                                                                                                                                                                                                        24
other: see 'Remark'                                                                                                                                                                                                                        15
other:                                                                                                                                                                                                                                      7
other: 100 ppm was the highest concentration tes

In [11]:

#%%
'''    
    1. critical effect가 우선, not specified는 
    2. effect level의 경우 소문자화한 다음 'no '(띄어쓰기 중요)가 있으면 0 / reduce, decrease, increase등이 있으면 1
    3. 이것도 없다면,,, Clinical signs, Mortality, Body weight and weight changes
    
'''

def find_colon(string):
    return ':' in string


def check_nan(string):
    return string == string
        

# result['Route of administration'].unique()
# result['Route of administration'].value_counts()

# effect 항목의 결과 내용 중 포함이 될 경우
def effective(string):
    if check_nan(string):
        string = string.lower()
        if 'no' in string and 'not' not in string:
            type_ = 0
        
        elif 'yes' in string or 'effects observed, treatment-related' in string:
            type_ = 1
            
        else:
            type_ = np.nan
    
    else:
        type_ = np.nan
        
    return type_

def conclusion(string):
    if check_nan(string):
        string = string.lower()
        #주요 단어를 적는 곳인듯
        if 'no' in string or 'absence' in string or 'not' in string:
            type_ = 0
        
        elif 'reduce' in string or 'decrease' in string or 'increase' in string or 'reduction' in string or 'adverse' in string or 'gain' in string:
            type_ = 1
        
        else:
            type_ = np.nan
    
    else:
        type_ = np.nan
        
    return type_

#%%
df = result.copy()

# effect
df['target1'] = df['Critical effects'].map(lambda x: effective(x))
df['target3'] = df['Clinical_sign'].map(lambda x: effective(x))

#effect level
df['target2'] = df['effect level'].map(lambda x: conclusion(x))

# make for result
df = df[['CasRN','target1','Relevant_human','target2','target3']]

In [13]:
# conclude target
df_null = df.isnull()
for i in range(len(df)):
    if df_null['target1'][i] == False:
        target = df['target1'][i]
    elif df_null['target2'][i] == False:
        target = df['target2'][i]
    elif df['target3'][i] == 0:
        target = df['target3'][i]
    else : target = np.nan
    df.loc[i,'target'] = target

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 165 entries, 0 to 164
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   CasRN           165 non-null    object 
 1   target1         28 non-null     float64
 2   Relevant_human  10 non-null     object 
 3   target2         90 non-null     float64
 4   target3         139 non-null    float64
 5   target          136 non-null    float64
dtypes: float64(4), object(2)
memory usage: 7.9+ KB


In [15]:
df['target'].value_counts()

0.0    94
1.0    42
Name: target, dtype: int64

In [16]:
# save df
df.to_csv('tg422_inhalation_0raw.csv', header = True, index = False)

# smiles 추가

In [24]:
import cirpy
import openpyxl
import warnings

import pandas as pd
import numpy as np

from tqdm import tqdm

pd.set_option('mode.chained_assignment', None)
warnings.filterwarnings("ignore")

In [25]:
data = pd.read_csv('tg422_inhalation_0raw.csv')
len(data['CasRN'].unique())

82

In [26]:
data.head()

CasRN  target1 Relevant_human  target2  target3  target
0   1873-88-7      NaN            NaN      0.0      0.0     0.0
1  68955-28-2      NaN            NaN      NaN      0.0     0.0
2  68477-39-4      NaN            NaN      0.0      NaN     0.0
3  68477-39-4      NaN            NaN      0.0      NaN     0.0
4    108-03-2      NaN            NaN      NaN      0.0     0.0

In [27]:
data['target'].unique()
data['target'].isna().sum()

data = data[data['target'].notna()]

casrn_na_idx = data[data['CasRN'] == '-'].index

data = data.drop(casrn_na_idx).reset_index(drop = True)

target_na_idx = data[data['target'] == ''].index

data = data.drop(target_na_idx).reset_index(drop=True)

In [28]:
data = data.drop_duplicates()
len(data)

65

In [29]:
tqdm.pandas()
data['SMILES'] = data.CasRN.progress_apply(lambda x: cirpy.resolve(x, 'smiles'))

100%|███████████████████████████████████████████████████████████████████████████████████| 65/65 [01:22<00:00,  1.28s/it]


In [30]:
data.to_csv('tg422_inhalation_1smiles.csv', header=True, index=False)

In [31]:
data.SMILES.isna().sum()
data = data[data['SMILES'].notna()].reset_index(drop = True)

In [32]:
len(data)

44

In [33]:
data.to_csv('tg422_inhalation_2deletenull.csv', header=True, index=False)